$$\require{mhchem}$$

# Adiabatic Flame Temperature: Full Equilibrium with Cantera

The adiabatic flame temperature is the temperature achieved when a given fuel-oxidizer combination is combusted in a perfectly insulated reactor. If no energy is lost to the surroundings via heat transfer and the reaction occurs at constant pressure, conservation of energy dictates that the total enthalpy of the reactants is equal to the total enthalpy of the products. Defining _state 1_ as the reactants and _state 2_ as the products, we can write:

$$h_{\text{mix}, 2} = h_{\text{mix}, 1}$$

The enthalpy of the mixture has two components:

1. the enthalpy of formation: $\Delta^{\circ}_{\text{f}}h$ 
2. the sensible enthalpy: $h_i$

To make this more concrete, let's pick a specific reaction: stoichiometric combustion of methane, $\ce{CH4}$ with air:

$$\ce{CH4 + 2 O2 + $7.52$N2 <=> 2 H2O + CO2 + $7.52$N2}$$

From the reaction stoichiometry (note that the net stoichiometric coefficient for $\ce{N2}$ is zero), the heat of combustion released by this reaction is:

$$\begin{aligned}q_{\text{rxn}} &= -\sum_k \nu_k \Delta^{\circ}_{\text{f}}h_k\\
&= \Delta^{\circ}_{\text{f}}h_{\rm CH_4} + 2\Delta^\circ_{\rm f}h_{\rm O_2} - \Delta^\circ_{\rm f}h_{\rm CO_2}-2 \Delta^\circ_{\rm f}h_{\rm H_2O}\end{aligned}$$

where $\nu_k$ is the stoichiometric coefficient of species $k$, which is positive for products and negative for reactants. By definition, the heat of reaction is released at the reference temperature.

However, since the reaction must occur at constant enthalpy, the heat released must be stored in the products. In essence, the heat is used to heat the products from the initial, reference, temperature to a final temperature:

$$\begin{aligned}q_{\text{heating}} &= \sum_k\nu_{\text{prod},k}\left[h_k\left(T_2\right) - h_k\left(T_1\right)\right]\\
&= h_{\ce{CO2}}\left(T_2\right) - h_{\ce{CO_2}}\left(T_1\right) + 2\left[h_{\ce{H2O}}\left(T_2\right) - h_{\ce{H2O}}\left(T_1\right)\right] + 7.52\left[h_{\ce{N2}}\left(T_2\right) - h_{\ce{N_2}}\left(T_1\right)\right]\end{aligned}$$

where, again, states 1 and 2 represent the final and initial states, respectively. $\nu_{\text{prod}}$ is the product stoichiometric coefficient. Note that this assumes complete combution - there are no reactant species left to heat.

For a purely adiabatic reaction, we will have $q_{\rm rxn} = q_{\rm heating}$, and hence:

$$\sum_k\nu_{\text{prod},k}\left[h_k\left(T_2\right) - h_k\left(T_1\right)\right] + \sum_k \nu_k \Delta^{\circ}_{\text{f}}h_k = 0$$

or:

$$h_{\ce{CO2}}\left(T_2\right) - h_{\ce{CO_2}}\left(T_1\right) + 2\left[h_{\ce{H2O}}\left(T_2\right) - h_{\ce{H2O}}\left(T_1\right)\right] + 7.52\left[h_{\ce{N2}}\left(T_2\right) - h_{\ce{N_2}}\left(T_1\right)\right] + \Delta^{\circ}_{\text{f}}h_{\rm CH_4} + 2\Delta^\circ_{\rm f}h_{\rm O_2} - \Delta^\circ_{\rm f}h_{\rm CO_2}-2 \Delta^\circ_{\rm f}h_{\rm H_2O} = 0$$

Note that only a few variables are dependent on the final temperature $T_2$.  We might be tempted to write the above equation in terms of those variables, but let's hold off for now. Our next step really depends on what method we use to solve the problem. In short, we need a means of calculating the enthalpy values, so that we may determine which value of $T_2$ satisfies our adiabatic condition.  

## Cantera - Full Equilibrium

In [another Notebook](./ad-flame-T-ct-comp-comb.ipynb), you learned how to find the adiabatic flame temperature assuming complete combustion. In that Notebook, the input file was restricted to two product species, plus nitrogen. In real combustion applications, of course, a wide array of minor product species and side reactions are possible that can impact $T_{ad}$ in ways that we might care about.

In general, performing calculations with Cantera involves three basic steps:

1. Identify or create a YAML-based input file.
2. Create Cantera phases and set initial conditions.
3. Run the calculation.

The only step requiring modification as compared to the [complete combustion Notebook](./ad-flame-T-ct-comp-comb.ipynb) is the first! We need a new input file to represent a more complex combustion chemistry. This is one of the most wonderfully powerful aspects of Cantera. You can simply change the input file without changing any of your code to see the effect of different assumptions on your calculation.

### Cantera input files, revisited

#### Mechanism files that come with Cantera.

From our previous input file, `methane.yaml`, we know that Cantera comes with a small collection of pre-defined iput files, and that one of these is `gri30.yaml`, which represents the GRI-3.0 mechanism for natural gas combustion developed in the 1970s and last updated in the 1990s.  In most cases, these mechanism files are included in Cantera for demonstration purposes only, and are not meant as an endorsement of that particular chemistry (this is particularly true in the case of `gri30.yaml`).

New and improved mechanisms are constantly developed and reported in the literature, representing the best-available data and community understanding. Many of these mechanisms are published in a format that is incompatible with Cantera, however. How can we use these input files in Cantera?

#### Conversion from Chemkin format

Most commonly, mechanisms are published and made available in a format consistent with the CHEMKIN software suite.  These input files can be converted to Cantera format, using the script `ck2yaml` (`ck` = "Chemkin").

We have downloaded the mechanism files for one such mechanism, the `NUIGMech1.1` from the National University of Ireland, Galway (http://www.nuigalway.ie/combustionchemistrycentre/mechanismdownloads/). These files are stored in the folder `input-files/NUIG`.

We will use the `ck2yaml` Python script file from the command line to convert these files into YAML format. Navigate to the directory `input-files/NUIG` and run the code:

```ck2yaml --input=NUIGMECH_1.1_HT.MECH, --thermo=NUIGMech1.1.THERM, --transport=NUIGMech1.1.TRAN --output=nuig_11.yaml```

This will result in several error messages and a failure to convert.  These mechanism files have hundreds of species and sometimes thousands of reactions; errors can be quite common! We have catalogued some of the most common conversion errors, so that you can debug as necessary, available [here](https://cantera.org/tutorials/ck2yaml-tutorial.html#debugging-common-errors-in-ck-files).

One thing we can try is re-running the conversion script with the `permissive` option:

```ck2yaml --input=NUIGMECH_1.1_HT.MECH, --thermo=NUIGMech1.1.THERM, --transport=NUIGMech1.1.TRAN --output=nuig_11.yaml --permissive```

We see that the conversion now runs successfully, but with a _signiciant_ number of warnings, species skipped, etc.  It is certainly worth following up on these and investigating, for any research endeavor of import!  But for today's demonstration, we will boldly forge on...